In [1]:
import os 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import  RunnablePassthrough
from typing import List
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from huggingface_hub import login
from dotenv import load_dotenv
import json
import warnings

load_dotenv()

# login()

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")


In [2]:
# os.environ['NEO4J_URI'] = 'neo4j+s://0a95ce3e.databases.neo4j.io'
# os.environ['NEO4J_USERNAME'] = 'neo4j'
# os.environ['NEO4J_PASSWORD'] = '1SgL9cPF7fKii-Tmrm231WbxAgxZ3Q-0mfnsHMriRoI'
# os.environ['NEO4J_DATABASE'] = 'neo4j'
# os.environ['AURA_INSTANCEID'] = '0a95ce3e'
# os.environ['AURA_INSTANCENAME'] = 'Instance01'

graph = Neo4jGraph()

In [3]:
# from pypdf import PdfMerger

# pdfs = ['wsu_courses.pdf','international_guide.pdf','international.pdf','english_requirements.pdf']

# merger = PdfMerger()

# for pdf in pdfs:
#     merger.append(pdf)

# merger.write("wsu-data.pdf")
# merger.close()

In [4]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("wsu-data.pdf")
# documents = loader.load()

In [5]:
# # a simple function that removes \n newline from the content
# def remove_ws(d):
#     text = d.page_content.replace('\n','')
#     d.page_content = text
#     return d

# # applied on the docs
# documents = [remove_ws(d) for d in documents]

In [6]:
# from langchain_experimental.graph_transformers import LLMGraphTransformer

# llm_transformer = LLMGraphTransformer(llm=llm)
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [7]:
# import pickle

# # Save graph_documents as a Pickle file
# with open("graph_enhanced.pkl", "wb") as file:
#     pickle.dump(graph_documents, file)

In [8]:
import pickle

# Open the saved graph documents
with open('graph_enhanced.pkl', 'rb') as file:
    graph_documents = pickle.load(file)

In [9]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [10]:
# directly show the graph resulting from the given Cypher query:
# default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

# def showGraph(cypher: str = default_cypher):
#     driver = GraphDatabase.driver(
#         uri = os.environ["NEO4J_URI"],
#         auth = (os.environ["NEO4J_USERNAME"],
#                 os.environ["NEO4J_PASSWORD"]))
#     session = driver.session()
#     widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
#     widget.node_label_mapping = 'id'
#     return widget

# showGraph()

In [12]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(model = "text-embedding-3-large"),
    #OllamaEmbeddings(model = "mxbai-embed-large"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [24]:
vector_index.similarity_search(query = "Liverpool City campus")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

[]

In [25]:
graph.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 2,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Document'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 10, 5, 5, 13, 33, 230000000, tzinfo=<UTC>),
  'readCount': 1}]

In [26]:
graph.refresh_schema()

In [27]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
# Entities_converted = convert_to_openai_function(Entities)
structured_llm = llm.with_structured_output(Entities)
entity_chain = prompt | structured_llm

# # Function to ensure entities.names is a list
# def ensure_list(entities_names):
#     if isinstance(entities_names, str):
#         return json.loads(entities_names)
#     return entities_names

In [28]:
entity_chain.invoke({"question": "Western Sydney"}).names

['Western Sydney']

In [29]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.
    
    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and combines them
    using the AND operator, ensuring exact matches without allowing misspellings.
    """
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join(words)
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    if not any(index["name"] == "entity" for index in graph.query("SHOW INDEXES")):
        graph.query("CREATE FULLTEXT INDEX entity FOR (n:__Entity__) ON EACH [n.id]")
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [30]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document". join(vector_data)}
    """
    return final_data

In [31]:
from langchain_core.runnables import  RunnablePassthrough

parser = StrOutputParser()

template = """Answer the question based only on the following context:
{context}

Question: {question}
- You are a representative at Western Sydney University's recruitment department, your job is to provide accurate information, and convince the domestic \
and international students to enrol at the university.
- Use natural language.
- Get the exact entities and relationships.
- Refer closely to the context of the query. Connect relevant entities for more contextually aware response. 
- The response should be concise yet detailed enough.
- If there's no entity matching the query, return "There is no information in the database regarding what \
you are requesting. Perhaps you should check your spelling or try a different prompt."

Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | parser
)

In [32]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_retriever
)

Failed to write data to connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))
Failed to write data to connection IPv4Address(('0a95ce3e.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))
Failed to write data to connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))
Failed to write data to connection IPv4Address(('0a95ce3e.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687)))


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\demon\AppData\Local\sagemaker\sagemaker\config.yaml


In [35]:
# print(graph_retriever("Dibya Shree Chhetry"))
 
query = str(input("Enter your query:"))
# print(graph_retriever(query))
answer = (chain.invoke(input = query))
retrieved_doc = compression_retriever.invoke(query)

In [34]:
def full_answer(a,b):
    return print(answer), retrieved_doc

full_answer(answer, retrieved_doc)

The Liverpool City campus of Western Sydney University offers students an exceptional educational experience with its state-of-the-art facilities and comprehensive academic programs. Located conveniently within the city, this campus is easily accessible for both domestic and international students. It boasts a diverse range of courses across various disciplines including business, engineering, health sciences, information technology, and more.

The campus features modern learning spaces designed to enhance student engagement and collaboration, as well as access to cutting-edge research facilities that provide hands-on experience in their respective fields. The academic staff at the Liverpool City campus are highly qualified professionals who are dedicated to fostering a supportive learning environment where students can thrive academically and personally.

In addition to its academic offerings, the campus provides numerous opportunities for students to engage with the community through

(None, [])